In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

C:\Users\sonph\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
df = pd.read_csv('C:/Users/sonph/OneDrive/Desktop/DUT_AI/PredictHousing/data/train_data.csv')
df

,Diện tích (m2),Hướng,Chiều ngang (m),Chiều dài (m),Số phòng ngủ,Số phòng vệ sinh,Số tầng,Tình trạng nội thất,Giá_per_m2,Tổng_phòng,Aspect_ratio,Diện_tích_per_phòng,Thành phố_encoded,Phường Xã_encoded,Loại hình_Nhà biệt thự,Loại hình_Nhà mặt phố mặt tiền,Loại hình_Nhà ngõ hẻm,Loại hình_Nhà phố liền kề,Giấy tờ pháp lý_encoded,Giá
0,46.0,Đông Bắc,3.76216,12.227019,3.0,3.0,1.000000,Không xác định,0.208696,6,0.307692,7.666667,6.460090,7.475523,False,True,False,False,4,9.60
1,160.0,Không xác định,8.00000,20.000000,3.0,2.0,2.962963,Hoàn thiện cơ bản,0.012500,5,0.400000,32.000000,2.702426,6.398009,False,True,False,False,4,2.00
2,60.0,Không xác định,4.60000,13.043478,7.0,5.0,1.000000,Không xác định,0.100000,12,0.352667,5.000000,6.460090,7.442779,False,True,False,False,4,6.00
3,27.0,Không xác định,3.00000,9.000000,3.0,3.0,4.000000,Không xác định,0.092593,6,0.333333,4.500000,6.460090,5.677360,False,False,False,True,3,2.50
4,44.9,Không xác định,4.00000,11.200000,2.0,2.0,2.000000,Hoàn thiện cơ bản,0.077728,4,0.357143,11.225000,6.460090,5.111358,False,False,True,False,4,3.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4768,36.0,Nam,3.60000,10.000000,2.0,2.0,2.000000,Không xác định,0.135556,4,0.360000,9.000000,6.460152,7.606696,False,False,True,False,4,4.88
4769,25.0,Không xác định,4.00000,6.500000,3.0,4.0,4.000000,Nội thất đầy đủ,0.200000,7,0.615385,3.571429,6.460152,7.578813,False,False,True,False,4,5.00
4770,61.0,Không xác định,4.40000,13.863636,2.0,2.0,1.000000,Không xác định,0.071311,4,0.317377,15.250000,4.122016,4.742985,False,False,True,False,4,4.35
4771,80.0,Bắc,5.00000,16.000000,1.0,1.0,1.000000,Không xác định,0.080000,2,0.312500,40.000000,6.460152,6.578365,False,False,True,False,4,6.40


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Diện tích (m2)                  4773 non-null   float64
 1   Hướng                           4773 non-null   object 
 2   Chiều ngang (m)                 4773 non-null   float64
 3   Chiều dài (m)                   4773 non-null   float64
 4   Số phòng ngủ                    4773 non-null   float64
 5   Số phòng vệ sinh                4773 non-null   float64
 6   Số tầng                         4773 non-null   float64
 7   Tình trạng nội thất             4773 non-null   object 
 8   Giá_per_m2                      4773 non-null   float64
 9   Tổng_phòng                      4773 non-null   int64  
 10  Aspect_ratio                    4773 non-null   float64
 11  Diện_tích_per_phòng             4773 non-null   float64
 12  Thành phố_encoded               47

In [19]:
df['Hướng'] = df['Hướng'].astype('category')
df['Tình trạng nội thất'] = df['Tình trạng nội thất'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4773 entries, 0 to 4772
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Diện tích (m2)                  4773 non-null   float64 
 1   Hướng                           4773 non-null   category
 2   Chiều ngang (m)                 4773 non-null   float64 
 3   Chiều dài (m)                   4773 non-null   float64 
 4   Số phòng ngủ                    4773 non-null   float64 
 5   Số phòng vệ sinh                4773 non-null   float64 
 6   Số tầng                         4773 non-null   float64 
 7   Tình trạng nội thất             4773 non-null   category
 8   Giá_per_m2                      4773 non-null   float64 
 9   Tổng_phòng                      4773 non-null   int64   
 10  Aspect_ratio                    4773 non-null   float64 
 11  Diện_tích_per_phòng             4773 non-null   float64 
 12  Thành phố_encoded   

In [20]:
x_train = df.drop('Giá', axis=1)
y_train = df['Giá']

In [21]:
def objective(trial):
    """
    Hàm này được Optuna gọi đi gọi lại nhiều lần.
    Mỗi lần gọi, 'trial' sẽ gợi ý một bộ tham số khác nhau.
    """
    
    # Định nghĩa không gian tìm kiếm (Search Space)
    param = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        'n_jobs': -1,
        
        # Các tham số cần tối ưu:
        # suggest_int: Gợi ý số nguyên
        # suggest_float: Gợi ý số thực (log=True giúp tìm tốt hơn ở dải giá trị nhỏ)
        'n_estimators': trial.suggest_int('n_estimators', 200, 2000),  # Tăng từ 100-1000
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),  # Mở rộng từ 0.005-0.2
        'num_leaves': trial.suggest_int('num_leaves', 15, 500),  # Tăng từ 20-300
        'max_depth': trial.suggest_int('max_depth', 3, 20),  # Tăng từ 3-15
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),  # Giảm từ 10-100
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),  # Mở rộng từ 0.5-1.0
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),  # Thêm tham số mới
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 100.0, log=True),  # Tăng từ 1e-8-10
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 100.0, log=True),  # Tăng từ 1e-8-10
    }

    try:
        # Khởi tạo mô hình với tham số vừa được gợi ý
        model = lgb.LGBMRegressor(**param)
        
        # Đánh giá bằng Cross-Validation (5-fold thay vì 3-fold)
        # scoring='neg_root_mean_squared_error' trả về số âm, cần thêm dấu trừ để lấy RMSE dương
        scores = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_root_mean_squared_error')
        rmse = -scores.mean()
        
        # Nếu kết quả là NaN, trả về một giá trị rất lớn (cho biết trial này xấu)
        if np.isnan(rmse):
            return 1e10
        
        # Optuna sẽ dựa vào giá trị này để biết bộ tham số tốt hay xấu
        return rmse
    except Exception as e:
        # Nếu có lỗi, trả về một giá trị rất lớn
        print(f"Error in trial: {e}")
        return 1e10

In [22]:
# Tạo một "nghiên cứu" (study), mục tiêu là tối thiểu hóa (minimize) lỗi RMSE
study = optuna.create_study(direction='minimize')

[I 2026-01-18 18:19:18,972] A new study created in memory with name: no-name-bd8cb44b-a7b4-43d7-9ca8-0e89d199d6c7


In [23]:
print("Đang bắt đầu quá trình tìm kiếm với Optuna...")
# n_trials=100: Thử 100 lần. Bạn có thể tăng lên 200 nếu máy mạnh.
study.optimize(objective, n_trials=20)

Đang bắt đầu quá trình tìm kiếm với Optuna...


[I 2026-01-18 18:19:25,697] Trial 0 finished with value: 0.41635564863517355 and parameters: {'n_estimators': 1303, 'learning_rate': 0.08311045568154891, 'num_leaves': 33, 'max_depth': 6, 'min_child_samples': 37, 'colsample_bytree': 0.5827609837975234, 'subsample': 0.5906193470210667, 'reg_alpha': 2.098311887116537, 'reg_lambda': 0.004528047672860853}. Best is trial 0 with value: 0.41635564863517355.
[I 2026-01-18 18:19:32,455] Trial 1 finished with value: 0.9188404971208562 and parameters: {'n_estimators': 943, 'learning_rate': 0.0026856427148950856, 'num_leaves': 276, 'max_depth': 19, 'min_child_samples': 29, 'colsample_bytree': 0.6268513596097331, 'subsample': 0.5920714019638449, 'reg_alpha': 0.5952876122782048, 'reg_lambda': 0.7324054229489043}. Best is trial 0 with value: 0.41635564863517355.
[I 2026-01-18 18:19:36,093] Trial 2 finished with value: 0.49350315518241983 and parameters: {'n_estimators': 630, 'learning_rate': 0.005966570680337138, 'num_leaves': 53, 'max_depth': 16, 'm

In [24]:
print(f"SỐ LẦN THỬ: {len(study.trials)}")
print(f"RMSE TỐT NHẤT (trên tập Valid): {study.best_value:.4f}")
print("THAM SỐ TỐT NHẤT:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

SỐ LẦN THỬ: 20
RMSE TỐT NHẤT (trên tập Valid): 0.3658
THAM SỐ TỐT NHẤT:
  n_estimators: 1671
  learning_rate: 0.08742634851160384
  num_leaves: 122
  max_depth: 6
  min_child_samples: 25
  colsample_bytree: 0.7088511699985144
  subsample: 0.46556137112429424
  reg_alpha: 0.10668718265547725
  reg_lambda: 3.004824001949854e-05


In [25]:
best_params = study.best_params

best_params['objective'] = 'regression'
best_params['metric'] = 'rmse'
best_params['random_state'] = 42
best_params['n_jobs'] = -1

In [26]:
final_model = lgb.LGBMRegressor(**best_params)
final_model.fit(x_train, y_train)

,boosting_type,'gbdt'
,num_leaves,122
,max_depth,6
,learning_rate,0.08742634851160384
,n_estimators,1671
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,25


In [29]:
df_test = pd.read_csv('C:/Users/sonph/OneDrive/Desktop/DUT_AI/PredictHousing/data/test_data.csv')
x_test = df_test.drop('Giá', axis=1)
y_test = df_test['Giá']
x_test['Hướng'] = x_test['Hướng'].astype('category')
x_test['Tình trạng nội thất'] = x_test['Tình trạng nội thất'].astype('category')
x_test.shape, y_test.shape

((1194, 19), (1194,))

In [30]:
y_pred_test = final_model.predict(x_test)

final_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
final_r2 = r2_score(y_test, y_pred_test)

print(f"RMSE trên tập Test: {final_rmse:.4f}")
print(f"R2 Score trên tập Test: {final_r2:.4f}")

RMSE trên tập Test: 0.3254
R2 Score trên tập Test: 0.9923


In [45]:
try:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()
except:
    print("Cần cài đặt plotly để xem biểu đồ: pip install plotly")

Cần cài đặt plotly để xem biểu đồ: pip install plotly


In [46]:
# Kiểm tra tương quan của các features với target
import matplotlib.pyplot as plt
correlations = x_train.corrwith(y_train).sort_values(ascending=False)
print("Top 10 features có tương quan cao nhất với giá:")
print(correlations.head(10))
print("\nThống kê Giá:")
print(y_train.describe())

Top 10 features có tương quan cao nhất với giá:
Diện tích (m2)                    0.328824
Số phòng ngủ                      0.250114
Phường Xã_encoded                 0.196323
Số phòng vệ sinh                  0.126728
Loại hình_Nhà biệt thự            0.108569
Loại hình_Nhà mặt phố mặt tiền    0.102137
Chiều ngang (m)                   0.096611
Thành phố_encoded                 0.092728
Chiều dài (m)                     0.070221
Số tầng                           0.057505
dtype: float64

Thống kê Giá:
count    5116.000000
mean        7.813273
std        20.858782
min         0.004400
25%         3.972500
50%         5.900000
75%         8.500000
max      1250.000000
Name: Giá (tỷ VNĐ), dtype: float64
